In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

In [28]:
df = pd.read_csv("/Users/safiaread/Downloads/calories.csv")
df['Gender'] = df['Gender'].map({'male': 0, 'female': 1})
df.drop(['User_ID'], axis=1, inplace=True)

In [29]:
X = df.drop(columns = ["Calories"])
y = df['Calories']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)  

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

y_scaler = StandardScaler()

# Fit & transform `y_train`, but only transform `y_test`
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

In [9]:
df.describe()

,User_ID,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
count,1.500000e+04,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,1.497736e+07,42.789800,174.465133,74.966867,15.530600,95.518533,40.025453,89.539533
std,2.872851e+06,16.980264,14.258114,15.035657,8.319203,9.583328,0.779230,62.456978
min,1.000116e+07,20.000000,123.000000,36.000000,1.000000,67.000000,37.100000,1.000000
25%,1.247419e+07,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000,35.000000
50%,1.499728e+07,39.000000,175.000000,74.000000,16.000000,96.000000,40.200000,79.000000
75%,1.744928e+07,56.000000,185.000000,87.000000,23.000000,103.000000,40.600000,138.000000
max,1.999965e+07,79.000000,222.000000,132.000000,30.000000,128.000000,41.500000,314.000000


In [37]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

# Add constant to X_train and X_test
X_train_scaled_ols = sm.add_constant(X_train_scaled)  # ✅ Add intercept
X_test_scaled_ols = sm.add_constant(X_test_scaled)    # ✅ Add intercept

# Fit OLS model correctly using X_train_scaled_ols
model = sm.OLS(y_train_scaled, X_train_scaled_ols).fit()  # ✅ Use correct X

# Make predictions
y_pred = model.predict(X_test_scaled_ols)

# Calculate the test MSE
test_mse = mean_squared_error(y_test_scaled, y_pred)
print(f"Test MSE: {test_mse}")


Test MSE: 0.03414418817263655


In [32]:
import numpy as np
from sklearn.linear_model import Ridge,RidgeCV


ridge_cv = RidgeCV(alphas=np.logspace(-10, 10, 19), cv=10) # Use 10-fold cross validation
ridge_cv.fit(X_train_scaled, y_train_scaled)

best_lambda = ridge_cv.alpha_

ridge_model = Ridge(alpha=best_lambda)
ridge_model.fit(X_train_scaled, y_train_scaled)

y_pred_ridge = ridge_model.predict(X_test_scaled)

# Calculate the test MSE
test_mse_ridge = mean_squared_error(y_test_scaled, y_pred_ridge)
print(f"Test MSE with Ridge Regression (lambda chosen by CV): {test_mse_ridge}")


Test MSE with Ridge Regression (lambda chosen by CV): 0.034145790841181106


In [35]:
import numpy as np
from sklearn.linear_model import Lasso, LassoCV


lasso_cv = LassoCV(alphas=np.logspace(-10, 10, 19), cv=10) # Use 10-fold cross validation
lasso_cv.fit(X_train_scaled, y_train_scaled)

best_lambda = lasso_cv.alpha_

lasso_model = Lasso(alpha=best_lambda)
lasso_model.fit(X_train_scaled, y_train_scaled)

y_pred_lasso = lasso_model.predict(X_test_scaled)

# Calculate the test MSE
test_mse_lasso = mean_squared_error(y_test_scaled, y_pred_lasso)
print(f"Test MSE with Lasso Regression (lambda chosen by CV): {test_mse_lasso}")

Test MSE with Lasso Regression (lambda chosen by CV): 0.03414418818051375


In [36]:
from sklearn.linear_model import ElasticNet

# Initialize the Elastic Net model
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5, fit_intercept=True)  # alpha controls regularization, l1_ratio controls mix of Lasso/Ridge

# Fit the model on training data
elastic_net.fit(X_train_scaled, y_train_scaled)

# Predict on both training and testing sets
y_train_pred = elastic_net.predict(X_train_scaled)
y_test_pred = elastic_net.predict(X_test_scaled)

# Compute the Mean Squared Error (MSE)
mse_train = mean_squared_error(y_train_scaled, y_train_pred)
mse_test = mean_squared_error(y_test_scaled, y_test_pred)

print(f"Training MSE: {mse_train:.4f}")
print(f"Testing MSE: {mse_test:.4f}")

Training MSE: 0.4424
Testing MSE: 0.4653
